In [1]:
%%capture
!pip install --upgrade datasets evaluate sacrebleu sentencepiece git+https://github.com/csebuetnlp/normalizer
!pip install transformers

In [2]:
import pandas as pd
import torch
import unicodedata
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoTokenizer
import evaluate
import os
from typing import Tuple


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Preprocessing functions
def standardize_to_NFC(text_list):
    """Normalize the text to NFC form for consistent diacritic handling."""
    return [unicodedata.normalize('NFC', text) for text in text_list]

def filter_single_word_sentence(eng_sents, yor_sents):
    """Filter out sentences that are single words in either language."""
    eng_inds = set([i for i, sent in enumerate(eng_sents) if len(sent.split()) > 1])
    yor_inds = set([i for i, sent in enumerate(yor_sents) if len(sent.split()) > 1])
    common_inds = sorted(list(eng_inds & yor_inds))

    eng_filtered = [eng_sents[i] for i in common_inds]
    yor_filtered = [yor_sents[i] for i in common_inds]

    return eng_filtered, yor_filtered

In [5]:
def load(input_dir: str):
    """Load data from CSV, normalize and filter it."""
    # Example dataset paths
    train_file = os.path.join(input_dir, 'train.tsv')  # Adjust the path to your training data
    val_file = os.path.join(input_dir, 'dev.tsv')      # Adjust the path to your validation data
    test_file = os.path.join(input_dir, 'test.tsv')

    # Load the training data
    train_df = pd.read_csv(train_file, sep='\t', names=['English', 'Yoruba'])
    val_df = pd.read_csv(val_file, sep='\t', names=['English', 'Yoruba'])
    test_df = pd.read_csv(test_file, sep='\t', names=['English', 'Yoruba'])

    return train_df, val_df, test_df

def preprocess(train_df, val_df, test_df) -> Tuple[Dataset, Dataset, pd.DataFrame]:
    # Normalize Yorùbá sentences to NFC
    train_df['Yoruba'] = standardize_to_NFC(train_df['Yoruba'])
    val_df['Yoruba'] = standardize_to_NFC(val_df['Yoruba'])
    test_df['Yoruba'] = standardize_to_NFC(test_df['Yoruba'])  # Normalize test data

    # Filter out single-word sentences from training and validation data only
    train_en, train_yo = filter_single_word_sentence(train_df['English'], train_df['Yoruba'])
    val_en, val_yo = filter_single_word_sentence(val_df['English'], val_df['Yoruba'])

    # Create Hugging Face dataset from pandas DataFrame
    train_dataset = Dataset.from_pandas(pd.DataFrame({'en': train_en, 'yo': train_yo}))
    val_dataset = Dataset.from_pandas(pd.DataFrame({'en': val_en, 'yo': val_yo}))
    test_dataset = Dataset.from_pandas(test_df)  # No filtering for test data

    return train_dataset, val_dataset, test_dataset  # Return the unfiltered test dataset


In [6]:
input_dir = '/kaggle/input/english-to-yoruba/'
train_df, val_df, test_df = load(input_dir)

In [7]:
display(train_df.head(3))
display(val_df.head(3))
display(test_df.head(3))

,English,Yoruba
0,English,Yoruba
1,Unit 1: What is Creative Commons?,﻿Ìdá 1: Kín ni Creative Commons?
2,This work is licensed under a Creative Commons...,Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons Attribu...


,English,Yoruba
0,English,Yoruba
1,"We prepare the saddle, and the goat presents i...",A di gàárì sílẹ̀ ewúrẹ́ ń yọjú; ẹrù ìran rẹ̀ ni?
2,"You have been crowned a king, and yet you make...",A fi ọ́ jọba ò ń ṣàwúre o fẹ́ jẹ Ọlọ́run ni?


,English,Yoruba
0,English,Yoruba
1,Pending the time she would finally pack and go...,Títí di ìgbà tí ó máa fi kó ẹrù rẹ̀ lọ pátápát...
2,She knew how best she was going to take care o...,Ó mọ bí ó ṣe má a tọ́jú ara rẹ̀ àti Tinú.


In [8]:
display(train_df.head(3))
display(val_df.head(3))
display(test_df.head(3))

,English,Yoruba
0,English,Yoruba
1,Unit 1: What is Creative Commons?,﻿Ìdá 1: Kín ni Creative Commons?
2,This work is licensed under a Creative Commons...,Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons Attribu...


,English,Yoruba
0,English,Yoruba
1,"We prepare the saddle, and the goat presents i...",A di gàárì sílẹ̀ ewúrẹ́ ń yọjú; ẹrù ìran rẹ̀ ni?
2,"You have been crowned a king, and yet you make...",A fi ọ́ jọba ò ń ṣàwúre o fẹ́ jẹ Ọlọ́run ni?


,English,Yoruba
0,English,Yoruba
1,Pending the time she would finally pack and go...,Títí di ìgbà tí ó máa fi kó ẹrù rẹ̀ lọ pátápát...
2,She knew how best she was going to take care o...,Ó mọ bí ó ṣe má a tọ́jú ara rẹ̀ àti Tinú.


In [9]:
train_dataset, val_dataset, test_dataset = preprocess(train_df, val_df, test_df)

In [10]:
# Rename columns in the Hugging Face datasets
train_dataset = train_dataset.rename_column('en', 'input_text')
train_dataset = train_dataset.rename_column('yo', 'labels')

val_dataset = val_dataset.rename_column('en', 'input_text')
val_dataset = val_dataset.rename_column('yo', 'labels')

test_dataset = test_dataset.rename_column('English', 'input_text')
test_dataset = test_dataset.rename_column('Yoruba', 'labels')



In [11]:
# from transformers import pipeline, VitsModel, AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = "Davlan/m2m100_418M-eng-yor-mt"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [14]:

from transformers import pipeline, VitsModel, AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'facebook/nllb-200-distilled-600M'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

You are using a model of type m2m_100 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at facebook/nllb-200-distilled-600M and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer

In [13]:
# from transformers import pipeline, VitsModel, AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = 'google-t5/t5-3b'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [15]:
from normalizer import normalize
from torch.utils.data import Dataset, DataLoader
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.input_text = pd.Series(data['input_text']).apply(normalize).tolist()
        self.labels = pd.Series(data['labels']).apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }

In [16]:
# Modify the data collation process to handle PyTorch tensors correctly
class MyDataCollatorForSeq2Seq(DataCollatorForSeq2Seq):
    def __call__(self, features):
        batch = {}
        batch["input_ids"] = torch.stack([feature["input_ids"] for feature in features])
        batch["attention_mask"] = torch.stack([feature["attention_mask"] for feature in features])

        # Labels should be processed differently for PyTorch tensors
        if isinstance(features[0]["labels"], torch.Tensor):
            batch["labels"] = torch.stack([feature["labels"] for feature in features])
        else:
            # Convert the list of lists to a PyTorch tensor
            batch["labels"] = torch.tensor([feature["labels"] for feature in features])

        return batch

In [17]:
from transformers import Trainer, TrainingArguments

# Custom Trainer class to ensure tensors are contiguous during training
class CustomSeq2SeqTrainer(Trainer):
    def save_model(self, output_dir=None, **kwargs):
        """Override save_model to ensure all model tensors are contiguous before saving."""
        if output_dir is None:
            output_dir = self.args.output_dir
        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.contiguous()
        super().save_model(output_dir, **kwargs)

    def training_step(self, model, inputs):
        """Override training_step to ensure tensors are contiguous during gradient updates."""
        for param in model.parameters():
            if not param.is_contiguous():
                param.data = param.contiguous()

        return super().training_step(model, inputs)

In [18]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_dataset, tokenizer)
val_dataset = Seq2SeqDataset(val_dataset, tokenizer)
test_dataset = Seq2SeqDataset(test_dataset, tokenizer)
# validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
val_dataloader = DataLoader(val_dataset, batch_size=32) #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=32) #batch_size=32
# validation_dataloader = DataLoader(validation_dataset, batch_size=32) #batch_size=32

In [19]:
# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    eps=1e-8,
    weight_decay=0.01,
)

In [20]:
# Define the TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=5,
    gradient_accumulation_steps=8,
    evaluation_strategy="steps",
    save_total_limit=0,
    eval_steps=50,
    save_steps=15000,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=False,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=100,
    weight_decay=0.01,
    #logging_dir='D:\\Datasets\\Thesis Data Test',
    logging_steps=20,

)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:

# Create a data collator for sequence-to-sequence tasks
data_collator = MyDataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=False,
    max_length=80,
    label_pad_token_id=tokenizer.pad_token_id,
)

In [22]:
# Create Trainer
trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=(custom_optimizer, None),
)

In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
50,11.551900,11.110100


In [ ]:
# Correct directory paths
model_output_dir = "/content/model/"
tokenizer_output_dir = "/content/model/"

# Save the model to the specified directory
model.save_pretrained(model_output_dir)

# Save the tokenizer to the specified directory
tokenizer.save_pretrained(tokenizer_output_dir)

print(f"Model saved to {model_output_dir}")
print(f"Tokenizer saved to {tokenizer_output_dir}")


In [ ]:
from tqdm import tqdm
import torch

def evaluate_model(model, tokenizer, eval_dataloader, device, max_samples=5):
    model.eval()  # Set model to evaluation mode
    model.to(device)

    predictions = []
    references = []
    num_samples = 0  # Initialize a counter

    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            # Move batch to device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Generate translations
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=80,  # Adjust max_length according to your data
                num_beams=5,    # Beam search for better results
                early_stopping=True
            )

            # Decode predictions and references
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

            # Calculate how many new samples to add
            remaining_samples = max_samples - num_samples

            # Store results, ensuring not to exceed the max_samples limit
            predictions.extend(decoded_preds[:remaining_samples])
            references.extend(decoded_labels[:remaining_samples])

            # Update the count of total samples collected
            num_samples += len(decoded_preds)

            # Stop when we have evaluated the specified number of samples
            if num_samples >= max_samples:
                break

    return predictions[:max_samples], references[:max_samples]



In [ ]:
preds, references = evaluate_model(model, tokenizer, val_dataloader, device)


In [ ]:
preds

In [ ]:
i = 4

test = val_dataset[i]
print('English: ', tokenizer.decode(test['input_ids']))
print('Correct yoruba: ', tokenizer.decode(test['labels']))
print('Predicted yoruba: ', preds[i])

In [ ]:
# Load the BLEU metric for evaluation using the new library
bleu_metric = evaluate.load("bleu")

# Tokenize predictions and references
tokenized_predictions = [pred.split() for pred in preds]  # Tokenize each prediction
tokenized_references = [ref for ref in references]  # Tokenize each reference and wrap it in a list

# Format predictions and references for BLEU metric calculation
bleu_metric.add_batch(
    predictions=preds,
    references=tokenized_references
)

# Calculate BLEU score
bleu_score = bleu_metric.compute()
print(f"BLEU Score: {bleu_score['bleu'] * 100:.2f}")


In [ ]:
preds

In [ ]:
print(preds)
print(tokenized_references)

In [ ]:
from tqdm import tqdm
import evaluate

def evaluate_model(model, tokenizer, eval_dataloader, device):
    model.eval()  # Set model to evaluation mode
    model.to(device)

    predictions = []
    references = []

    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            # Move batch to device
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Generate translations
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=80,  # Adjust max_length according to your data
                num_beams=5,  # Beam search for better results
                early_stopping=True
            )

            # Decode predictions
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

            # Store results
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)

    return predictions, references

In [ ]:
predictions, references =  evaluate_model(model, tokenizer, val_dataloader, device)

In [ ]:
# Load the BLEU metric for evaluation using the new library
bleu_metric = evaluate.load("bleu")

# Format predictions and references for BLEU metric calculation
bleu_metric.add_batch(
    predictions=predictions,
    references=references
)

# Calculate BLEU score
bleu_score = bleu_metric.compute()
print(f"BLEU Score: {bleu_score['bleu'] * 100:.2f}")


In [ ]:
predictions, references =  evaluate_model(model, tokenizer, test_dataloader, device)

In [ ]:
# Display some sample results
for i in range(5):  # Display first 5 samples
    print(f"Input: {val_dataset.input_text[i]}")
    print(f"Prediction: {predictions[i]}")
    print(f"Reference: {references[i]}")
    print("-" * 30)


In [ ]:
pd.Series(predictions).to_csv('first_submission.csv')